<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Solutions/Python/pyspark_max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 60.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=50cb02d6c9b4c9c0659b7b148054763c18118fa4aa5dff0efe9e903b01f4acca
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("datediff")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [43]:
df = spark.read\
        .format('csv')\
        .option('sep','|')\
        .option('inferSchema',True)\
        .option('header',True)\
        .load('/content/pyspark_max.csv')

In [44]:
df.show()

+----------+-----------------+-------+------+---------+
|      Date|      Description| Credit| Debit|  Balance|
+----------+-----------------+-------+------+---------+
|12-01-2021|           Salary|87771.0|   0.0|87,771.00|
|12-01-2021| Previous_mon_bal|16922.1|   0.0|104693.10|
|12-01-2021|           Stocks|    0.0|5000.0| 99693.10|
|12-02-2021|   Post_office_rd|    0.0|2500.0| 97193.10|
|12-03-2021|          EB_bill|    0.0| 480.0| 96713.10|
|12-03-2021|             Chit|    0.0|5000.0| 91713.10|
|12-03-2021|              SIP|    0.0| 500.0| 91213.10|
|12-03-2021|           Refund| 1100.0|   0.0| 92313.10|
|12-04-2021|          dentist|    0.0| 600.0| 91713.10|
|12-04-2021|       eating_out|    0.0| 900.0| 90813.10|
|12-04-2021|    hair_medicine|    0.0|1722.0| 89091.10|
|12-04-2021|           petrol|    0.0| 400.0| 88691.10|
|12-04-2021|        vegtables|    0.0| 250.0| 88441.10|
|12-05-2021|         land_emi|    0.0|4000.0| 84441.10|
|12-05-2021|           snacks|    0.0| 220.0| 84

In [45]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,col,max,to_date
from datetime import datetime
from pyspark.sql.types import DecimalType

In [46]:
df = df.withColumn('Date',to_date('Date','MM-dd-yyyy'))\
.withColumn('Debit',col('Debit').cast(DecimalType()))

In [47]:
df.show()

+----------+-----------------+-------+-----+---------+
|      Date|      Description| Credit|Debit|  Balance|
+----------+-----------------+-------+-----+---------+
|2021-12-01|           Salary|87771.0|    0|87,771.00|
|2021-12-01| Previous_mon_bal|16922.1|    0|104693.10|
|2021-12-01|           Stocks|    0.0| 5000| 99693.10|
|2021-12-02|   Post_office_rd|    0.0| 2500| 97193.10|
|2021-12-03|          EB_bill|    0.0|  480| 96713.10|
|2021-12-03|             Chit|    0.0| 5000| 91713.10|
|2021-12-03|              SIP|    0.0|  500| 91213.10|
|2021-12-03|           Refund| 1100.0|    0| 92313.10|
|2021-12-04|          dentist|    0.0|  600| 91713.10|
|2021-12-04|       eating_out|    0.0|  900| 90813.10|
|2021-12-04|    hair_medicine|    0.0| 1722| 89091.10|
|2021-12-04|           petrol|    0.0|  400| 88691.10|
|2021-12-04|        vegtables|    0.0|  250| 88441.10|
|2021-12-05|         land_emi|    0.0| 4000| 84441.10|
|2021-12-05|           snacks|    0.0|  220| 84221.10|
|2021-12-0

In [48]:
window_spec = Window\
.partitionBy('Date')\
.orderBy(col('Debit').desc())

In [49]:
df = df.withColumn('max_exp_per_day',max(col('Debit')).over(window_spec))

In [50]:
df.show()

+----------+-----------------+-------+-----+---------+---------------+
|      Date|      Description| Credit|Debit|  Balance|max_exp_per_day|
+----------+-----------------+-------+-----+---------+---------------+
|2021-12-01|           Stocks|    0.0| 5000| 99693.10|           5000|
|2021-12-01|           Salary|87771.0|    0|87,771.00|           5000|
|2021-12-01| Previous_mon_bal|16922.1|    0|104693.10|           5000|
|2021-12-02|   Post_office_rd|    0.0| 2500| 97193.10|           2500|
|2021-12-03|             Chit|    0.0| 5000| 91713.10|           5000|
|2021-12-03|              SIP|    0.0|  500| 91213.10|           5000|
|2021-12-03|          EB_bill|    0.0|  480| 96713.10|           5000|
|2021-12-03|           Refund| 1100.0|    0| 92313.10|           5000|
|2021-12-04|    hair_medicine|    0.0| 1722| 89091.10|           1722|
|2021-12-04|       eating_out|    0.0|  900| 90813.10|           1722|
|2021-12-04|          dentist|    0.0|  600| 91713.10|           1722|
|2021-